In [19]:
import pandas as pd
from azure.storage.blob import BlobServiceClient
from io import BytesIO

# Azure configuration
storage_account_name = "stgipplgt10153"
sas_token = "sp=racwdl&st=2025-08-07T15:22:55Z&se=2026-08-06T23:37:55Z&spr=https&sv=2024-11-04&sr=c&sig=ks0K4JyQik%2FYo%2BiIXNTggfiJgSOCVWH3CI7yNNG9N7I%3D"
container_name = "stgippl-azrgt10153"
blob_folder = "uat/inbound/locationmaster/locationmaster_202508031910/"

# Create blob client and read data
blob_service_client = BlobServiceClient(
    account_url=f"https://{storage_account_name}.blob.core.windows.net",
    credential=sas_token
)

container_client = blob_service_client.get_container_client(container_name)
blobs = list(container_client.list_blobs(name_starts_with=blob_folder))
parquet_files = [blob.name for blob in blobs if blob.name.endswith('.parquet')]

# Read and combine all parquet files
all_dfs = []
for file_name in parquet_files:
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
    blob_data = blob_client.download_blob().readall()
    temp_df = pd.read_parquet(BytesIO(blob_data))
    all_dfs.append(temp_df)

# Combine and display results
df = pd.concat(all_dfs, ignore_index=True)
print(f"Loaded {len(parquet_files)} files with {df.shape[0]} rows and {df.shape[1]} columns")
df.head()

Loaded 1 files with 9037 rows and 41 columns


,Location,NetworkLocationDescription,LocationStatus,StoreFormat,LocationGroup,LocationType,StoreArea,SalesRegion,SalesRegionDescription,Market,...,Address,DC,StatLocation,ValidSubLocationFlag,SubLocation,AllLocation,IsNew,SellingLegalEntity,BillingCustomerID,TenantRegionIdentifier
0,CH10051851,Christ Aarau,Open For Sale,Standard,Store,White,NaN,EUMEA,"Europe, Middle East and Africa",CH,...,HINTERE VORSTADT 11,EDC,CH10051851,False,Undefined,All,None,CPCH,None,EUMEA
1,BR10060781,Pandora @ Pontal Shopping,Open For Sale,Standard,Store,Concept Store,53.0,AMER,Americas,BR,...,AVENIDA PADRE CACIQUE 2893. PRAIA DE BELAS,USDC,BR10060781,False,Undefined,All,None,BRA,BRT,AMER
2,TR10061472,Pandora City’s Nişantaşı,Open For Sale,Standard,Store,Concept Store,32.0,EUMEA,"Europe, Middle East and Africa",TR,...,TEŞVIKIYE MAH. TEŞVIKIYE CAD. NO:12,EDC,TR10061472,False,Undefined,All,None,TUR,901074,EUMEA
3,AM10062377,TIMELESS Yerevan Erebuni,Open For Sale,Kiosk,Store,Shop in Shop,13.0,EUMEA,"Europe, Middle East and Africa",AM,...,"17, 1 EREBUNI ST",None,AM10062377,False,Undefined,All,None,EEPL,None,EUMEA
4,DE10052749,Galeria Karstadt Kaufhof Freiburg,Open For Sale,Standard,Store,Shop in Shop,NaN,EUMEA,"Europe, Middle East and Africa",DE,...,KAISER-JOSEPH-STRASSE 165,EDC,DE10052749,False,Undefined,All,None,CPAN,None,EUMEA
